# Koalas - Simplifying PySpark with Pandas

This notebook provides a sample example of scaling your machine learning using [Koalas: Pandas API on Apache Spark](https://github.com/databricks/koalas).

<img src="https://files.training.databricks.com/images/fire-koala.jpg" width=150/>

Pandas is the de facto standard (single-node) dataframe implementation in Python, while Apache Spark is the de facto standard for big data processing. We can use Koalas to:
* Use Pandas (in lieu of PySpark) syntax
* Easily scale your machine learning utilizing Apache Spark with Pandas syntax

For more information:
* [Koalas Documentation](https://koalas.readthedocs.io/)
* [Koalas GitHub Repo](https://github.com/databricks/koalas)
* [koalas (PyPI)](https://pypi.org/project/koalas/)
* [10 minutes to Koalas](https://koalas.readthedocs.io/en/latest/getting_started/10min.html)

Dependencies:
* Please install `koalas` and `yellowbrick` Python libraries

In [2]:
import pandas as pd
import pyspark
import databricks.koalas as ks

## Data Source
We will start by reading in the [Kaggle Boston Housing dataset](https://www.kaggle.com/c/boston-housing/data).

![](https://storage.googleapis.com/kaggle-competitions/kaggle/5315/logos/front_page.png)

**Housing Values in Suburbs of Boston**
The medv variable is the target variable.

**Data description**
The Boston data frame has 506 rows and 14 columns.

This data frame contains the following columns:

| columns | description |
| :------- | :----------- |
| crim |  per capita crime rate by town. |
| zn | proportion of residential land zoned for lots over 25,000 sq.ft. |
| indus | proportion of non-retail business acres per town. |
| chas |  Charles River dummy variable (= 1 if tract bounds river; 0 otherwise). |
| nox |  nitrogen oxides concentration (parts per 10 million). |
| rm | average number of rooms per dwelling. |
| age |  proportion of owner-occupied units built prior to 1940. |
| dis |  weighted mean of distances to five Boston employment centres. |
| rad |  index of accessibility to radial highways. |
| tax | full-value property-tax rate per \$10,000. |
| ptratio | pupil-teacher ratio by town. |
| black | 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town. |
| lstat | lower status of the population (percent). |
| medv | median value of owner-occupied homes in \$1000s. |

Sources:
* Harrison, D. and Rubinfeld, D.L. (1978) Hedonic prices and the demand for clean air. J. Environ. Economics and Management 5, 81–102.
* Belsley D.A., Kuh, E. and Welsch, R.E. (1980) Regression Diagnostics. Identifying Influential Data and Sources of Collinearity. New York: Wiley.

Resources:
* A great more indepth blog post is Susan Li's [Building A Linear Regression with PySpark and MLlib](https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a)

### Download the Boston Housing dataset

In [5]:
%sh mkdir -p /dbfs/tmp/dennylee/samples/boston/ && wget -O /dbfs/tmp/dennylee/samples/boston/boston-housing.csv https://raw.githubusercontent.com/databricks/tech-talks/master/datasets/boston-housing.csv && ls -al /dbfs/tmp/dennylee/samples/boston/

--2020-05-08 03:10:25-- https://raw.githubusercontent.com/databricks/tech-talks/master/datasets/boston-housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35818 (35K) [text/plain]
Saving to: ‘/dbfs/tmp/dennylee/samples/boston/boston-housing.csv’

 0K .......... .......... .......... .... 100% 868K=0.04s

2020-05-08 03:10:25 (868 KB/s) - ‘/dbfs/tmp/dennylee/samples/boston/boston-housing.csv’ saved [35818/35818]

total 43
drwxrwxrwx 2 root root 4096 May 8 2020 .
drwxrwxrwx 2 root root 4096 May 7 15:21 ..
-rwxrwxrwx 1 root root 35818 May 8 2020 boston-housing.csv

In [6]:
# Configure file path
dbfs_path = '/tmp/dennylee/samples/boston/boston-housing.csv'

## Load Data Using Pandas Syntax via Koalas
We can load our data using the Pandas syntax `.read_csv()` to read the CSV file and use the Pandas syntax `.head()` to review the top 5 rows of our Spark DataFrame.

In [8]:
# Read CSV to pandas dataframe
pdf = pd.read_csv('/dbfs/tmp/dennylee/samples/boston/boston-housing.csv')

# Display it
pdf.head(10)

,ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9
5,11,0.22489,12.5,7.87,0,0.524,6.377,94.3,6.3467,5,311,15.2,392.52,20.45,15.0
6,12,0.11747,12.5,7.87,0,0.524,6.009,82.9,6.2267,5,311,15.2,396.90,13.27,18.9
7,13,0.09378,12.5,7.87,0,0.524,5.889,39.0,5.4509,5,311,15.2,390.50,15.71,21.7
8,14,0.62976,0.0,8.14,0,0.538,5.949,61.8,4.7075,4,307,21.0,396.90,8.26,20.4
9,15,0.63796,0.0,8.14,0,0.538,6.096,84.5,4.4619,4,307,21.0,380.02,10.26,18.2


In [9]:
# Convert pandas dataframe to Koalas DataFrame
kdf = ks.DataFrame(pdf)

# Display it
kdf.head(10)

,ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
96,142,1.62864,0.0,21.89,0,0.624,5.019,100.0,1.4394,4,437,21.2,396.90,34.41,14.4
97,143,3.32105,0.0,19.58,1,0.871,5.403,100.0,1.3216,5,403,14.7,396.90,26.82,13.4
98,146,2.37934,0.0,19.58,0,0.871,6.130,100.0,1.4191,5,403,14.7,172.91,27.80,13.8
99,148,2.36862,0.0,19.58,0,0.871,4.926,95.7,1.4608,5,403,14.7,391.71,29.53,14.6
100,149,2.33099,0.0,19.58,0,0.871,5.186,93.8,1.5296,5,403,14.7,356.99,28.32,17.8
101,150,2.73397,0.0,19.58,0,0.871,5.597,94.9,1.5257,5,403,14.7,351.85,21.45,15.4
102,151,1.65660,0.0,19.58,0,0.871,6.122,97.3,1.6180,5,403,14.7,372.80,14.10,21.5
103,154,2.14918,0.0,19.58,0,0.871,5.709,98.5,1.6232,5,403,14.7,261.95,15.79,19.4
104,155,1.41385,0.0,19.58,1,0.871,6.129,96.0,1.7494,5,403,14.7,321.02,15.12,17.0
105,157,2.44668,0.0,19.58,0,0.871,5.272,94.0,1.7364,5,403,14.7,88.63,16.14,13.1


## Exploratory Data Analysis
We will use a combination of `display` and `koalas` to review our data.

### Determine Possible Linear Correlation Between Multiple Independent Variables
Let's start off by creating a Scatterplot matrix of the different variables by using Databricks `display()`

In [12]:
def displayKoalas(df):
  display(df.toPandas())

In [13]:
# View Scatterplot of data
displayKoalas(kdf)

ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0,0.5379999999999999,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6
4,0.032369999999999996,0.0,2.18,0,0.45799999999999996,6.997999999999999,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.45799999999999996,7.147,54.2,6.0622,3,222,18.7,396.9,5.33,36.2
7,0.08829,12.5,7.87,0,0.524,6.0120000000000005,66.6,5.5605,5,311,15.2,395.6,12.43,22.9
11,0.22489,12.5,7.87,0,0.524,6.377000000000001,94.3,6.3467,5,311,15.2,392.52,20.45,15.0
12,0.11747,12.5,7.87,0,0.524,6.0089999999999995,82.9,6.2267,5,311,15.2,396.9,13.27,18.9
13,0.09378,12.5,7.87,0,0.524,5.888999999999999,39.0,5.4509,5,311,15.2,390.5,15.71,21.7
14,0.62976,0.0,8.14,0,0.5379999999999999,5.949,61.8,4.7075,4,307,21.0,396.9,8.26,20.4
15,0.6379600000000001,0.0,8.14,0,0.5379999999999999,6.096,84.5,4.4619,4,307,21.0,380.02,10.26,18.2


#### `rm` and `medv`
In the above scatterplot, you can see a possible correlation between `rm` (average rooms per dwelling) and `medv` (median value of owner-occupied homes).  This can be better seen in the following scatterplot of just these two variables.

In [15]:
# View Scatterplot of data
displayKoalas(kdf)

ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0,0.5379999999999999,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6
4,0.032369999999999996,0.0,2.18,0,0.45799999999999996,6.997999999999999,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.45799999999999996,7.147,54.2,6.0622,3,222,18.7,396.9,5.33,36.2
7,0.08829,12.5,7.87,0,0.524,6.0120000000000005,66.6,5.5605,5,311,15.2,395.6,12.43,22.9
11,0.22489,12.5,7.87,0,0.524,6.377000000000001,94.3,6.3467,5,311,15.2,392.52,20.45,15.0
12,0.11747,12.5,7.87,0,0.524,6.0089999999999995,82.9,6.2267,5,311,15.2,396.9,13.27,18.9
13,0.09378,12.5,7.87,0,0.524,5.888999999999999,39.0,5.4509,5,311,15.2,390.5,15.71,21.7
14,0.62976,0.0,8.14,0,0.5379999999999999,5.949,61.8,4.7075,4,307,21.0,396.9,8.26,20.4
15,0.6379600000000001,0.0,8.14,0,0.5379999999999999,6.096,84.5,4.4619,4,307,21.0,380.02,10.26,18.2


#### `lstat` and `medv`
In the above scatterplot, you can see a possible negative linear correlation between `lstat` (lower status of population) and `medv` (median value of owner-occupied homes).  This can be better seen in the following scatterplot of just these two variables.

In [17]:
# View Scatterplot of data
displayKoalas(kdf)

ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0,0.5379999999999999,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6
4,0.032369999999999996,0.0,2.18,0,0.45799999999999996,6.997999999999999,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.45799999999999996,7.147,54.2,6.0622,3,222,18.7,396.9,5.33,36.2
7,0.08829,12.5,7.87,0,0.524,6.0120000000000005,66.6,5.5605,5,311,15.2,395.6,12.43,22.9
11,0.22489,12.5,7.87,0,0.524,6.377000000000001,94.3,6.3467,5,311,15.2,392.52,20.45,15.0
12,0.11747,12.5,7.87,0,0.524,6.0089999999999995,82.9,6.2267,5,311,15.2,396.9,13.27,18.9
13,0.09378,12.5,7.87,0,0.524,5.888999999999999,39.0,5.4509,5,311,15.2,390.5,15.71,21.7
14,0.62976,0.0,8.14,0,0.5379999999999999,5.949,61.8,4.7075,4,307,21.0,396.9,8.26,20.4
15,0.6379600000000001,0.0,8.14,0,0.5379999999999999,6.096,84.5,4.4619,4,307,21.0,380.02,10.26,18.2


#### Use Pandas `.corr` to Calculate Correlation Coefficents 
We can quickly calculate the correlation matrix of all attributes with `medv` using Pandas `.corr`.

In [19]:
# Calculate using Pandas `corr`
pdf_corr = kdf.toPandas().corr()

# Add Index 
pdf_corr['index1'] = pdf_corr.index

# Display values related to `medv`
display(pdf_corr.loc[:, ['index1', 'medv']])

index1,medv
ID,-0.2216941865161163
crim,-0.4074543235732594
zn,0.34484197569664343
indus,-0.47393197065920345
chas,0.20438998859918686
nox,-0.41305415199207746
rm,0.6895980892872157
age,-0.35888827406190177
dis,0.24942226829396363
rad,-0.35225082424563325


### Use Koalas for Pandas Syntax
* Drop rows with missing values via `.dropna()`
* Rename columns using `.columns`

In [21]:
# drop rows with missing values
kdf = kdf.dropna()  
kdf.count()

Out[11]: ID 333
crim 333
zn 333
indus 333
chas 333
nox 333
rm 333
age 333
dis 333
rad 333
tax 333
ptratio 333
black 333
lstat 333
medv 333
Name: 0, dtype: int64

In [22]:
# New column names
column_names = ['ID', 'crime', 'zone', 'industry', 'bounds_river', 'nox', 'rooms', 'age', 'distance', 'radial_highway', 'tax', 'pupil_teacher', 'black_proportion', 'lower_status', 'median_value']

# Rename columns
kdf.columns = column_names

### Choosing Features
Reviewing the correlation coefficient matrix and scatterplots, let's choose features that have slighty stronger positive or negative correlation to the `median_value` where `abs(correlation coefficients) >= 0.4`.

In [24]:
# All columns
featureColumns = ['ID', 'crime', 'zone', 'industry', 'bounds_river', 'nox', 'rooms', 'age', 'distance', 'radial_highway', 'tax', 'pupil_teacher', 'black_proportion', 'lower_status', 'median_value']

# # Limit feature columns to abs(correlation coefficients) >= 0.4 
# featureColumns = ['crime', 'industry', 'nox', 'rooms', 'tax', 'pupil_teacher', 'lower_status']

# # All columns but median_value
# featureColumns = ['ID', 'crime', 'zone', 'industry', 'bounds_river', 'nox', 'rooms', 'age', 'distance', 'radial_highway', 'tax', 'pupil_teacher', 'black_proportion', 'lower_status']

## Build our Linear Regression Model

Let's build our Linear Regression model using Scikit-Learn.

In [26]:
# Re-generate Pandas DataFrame from updated Koala DataFrame
pdf = kdf.toPandas()

# Split training and test datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pdf[featureColumns].values, pdf['median_value'].values, test_size=0.20, random_state=567248)

## Evaluate Model Performance
Calculate [RMSE](https://en.wikipedia.org/wiki/Root-mean-square_deviation) and [r<sup>2</sup>](https://en.wikipedia.org/wiki/Coefficient_of_determination)

In [28]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
import math

# Create linear regression object
lr = Ridge()
#lr = Lasso()

# Train the model using the training sets
lr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = lr.predict(X_test)

# Calculate RMSE
print("RMSE: %.3f" % math.sqrt(mean_squared_error(y_test, y_pred)))
print("R^2: %.3f" % r2_score(y_test, y_pred))

RMSE: 0.001
R^2: 1.000

### Prediction Error Plot

In [30]:
from yellowbrick.regressor import PredictionError

# Instantiate the linear model and visualizer
model = Ridge()
visualizer_pe = PredictionError(model, size=(1000,800))

visualizer_pe.fit(X_train, y_train)  # Fit the training data to the visualizer
visualizer_pe.score(X_test, y_test)  # Evaluate the model on the test data
visualizer_pe.poof()                 # Finalize and render the figure

### Residuals Error Plot

In [32]:
from yellowbrick.regressor import ResidualsPlot

# Instantiate the linear model and visualizer
ridge = Ridge()
visualizer_re = ResidualsPlot(ridge, size=(1000, 800))

visualizer_re.fit(X_train, y_train)  # Fit the training data to the model
visualizer_re.score(X_test, y_test)  # Evaluate the model on the test data
visualizer_re.poof()                 # Draw/show/poof the data

## Discussion
While this is a small dataset example, as you increase the iterations and/or data sizes by using `koalas`.  While there is great functionality with the `pandas` syntax, it can be at times very slow.  But with `koalas`, you can easily scale the same code while using PySpark in the backend.